In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/sarvesh-assignment/flux_plain.dat', sep='\t',skiprows=18)
data

In [ ]:
data.to_csv('flux plain.csv')

In [ ]:
data = pd.read_csv('/kaggle/working/flux plain.csv')
data

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data = data.head(284)
data

In [ ]:
# Assuming your DataFrame columns have spaces in the column names, remove the extra spaces.
data.rename(columns={'Flux    ': 'Flux', '!Time    ': 'Time', 'T_+ve   ':'T_+ve','T_-ve   ':'T_-ve','Fluxpos ':'Fluxpos'}, inplace=True)

# Convert the 'Flux' and 'Time' columns to float
data['Flux'] = pd.to_numeric(data['Flux'], errors='coerce')  # 'coerce' to handle non-numeric values
data['Time'] = pd.to_numeric(data['Time'], errors='coerce')
data['T_+ve'] = pd.to_numeric(data['T_+ve'], errors='coerce')  # 'coerce' to handle non-numeric values
data['T_-ve'] = pd.to_numeric(data['T_-ve'], errors='coerce')
data['Fluxpos'] = pd.to_numeric(data['Fluxpos'], errors='coerce')  # 'coerce' to handle non-numeric values
data['Fluxneg'] = pd.to_numeric(data['Fluxneg'], errors='coerce')
data['Fluxneg'] = np.abs(data['Fluxneg'])/10**10

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
# Extract data
x = data['Time']
y = data['Flux']
x_err = [np.abs(data['T_-ve']), data['T_+ve']]
y_err = [data['Fluxpos'],data['Fluxneg']]

# Create a scatter plot with error bars
plt.errorbar(x, y, xerr=x_err, yerr=y_err, fmt='o', markersize=5, color='red', ecolor='green', capsize=3)

# Set x-axis and y-axis to logarithmic scale
plt.xscale('log')
plt.yscale('log')

# Label axes and add a title
plt.xlabel('Time')
plt.ylabel('Flux')
plt.title('Flux vs Time with Error Bars (log-log scale)')

# Show the plot
plt.show()


In [ ]:
data.info()

In [ ]:
data.isna().sum()

# bidirection lstm Model

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
data = data.dropna()
# Check for missing or NaN values in your data
if data.isnull().values.any():
    raise ValueError("Data contains missing or NaN values.")

# Assuming 'data' contains your dataset
# X should contain all columns except 'Flux'
X = data.drop('Flux', axis=1).values.astype(np.float32)

# Use 'Flux' column as y
y = data["Flux"].values.astype(np.float32)

# Check if y contains any NaN values
if np.isnan(y).any():
    raise ValueError("Target values 'y' contain NaN values.")

# Normalize input data to the range [0, 1]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape the input data to have a sequence dimension
# Assuming you have only one feature per time step
X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape to match the input shape (samples, time steps, features)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train the Bidirectional LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.2), input_shape=(X.shape[1], 1)),
    tf.keras.layers.Dense(1)
])

# Reduce learning rate and add gradient clipping to avoid NaN
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=1.0)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Add early stopping to monitor validation loss and prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Print model summary for debugging
model.summary()

# Train the model and monitor progress
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Predict values
y_pred = model.predict(X_test)

In [ ]:
y_pred.reshape(-1)

In [ ]:
data = data.dropna()

In [ ]:
y_pred_x = model.predict(X)

In [ ]:
X

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
# Extract data
data = data.dropna()
x = data['Time']
y = y_pred.reshape(-1)[:282]
#x_err = [np.abs(data['T_-ve']), data['T_+ve']]
#y_err = [data['Fluxpos'],data['Fluxneg']]

# Create a scatter plot with error bars
#plt.errorbar(x, y, xerr=x_err, yerr=y_err, fmt='o', markersize=5, color='red', ecolor='green', capsize=3)
plt.scatter(x,y,color='red',marker='.',s=10)
# Set x-axis and y-axis to logarithmic scale
plt.xscale('log')
plt.yscale('log')

# Label axes and add a title
plt.xlabel('Time')
plt.ylabel('Flux')
plt.title('Flux vs Time with Error Bars (log-log scale)')

# Show the plot
plt.show()


In [ ]:
x.shape

In [ ]:
y.shape

# As I am not familier with the bidirection lstm model i am trying out some other methods and models top get my predictions right on point.

In [ ]:
x = data.drop('Flux', axis=1)
y = data['Flux']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=42)

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
h2o_frame = h2o.H2OFrame(data)
x = h2o_frame.columns
y = 'Flux'
x.remove(y)

In [ ]:
h2o_automl = H2OAutoML(sort_metric='mse', max_runtime_secs=7*60, seed=42)
h2o_automl.train(x=x, y=y, training_frame=h2o_frame)

In [ ]:
h2o_models = h2o.automl.get_leaderboard(h2o_automl, extra_columns = "ALL")
h2o_models

In [ ]:
from h2o.model.regression import h2o_mean_squared_error

h2o_frame_test = h2o.H2OFrame(data)

y_pred = h2o_automl.predict(h2o_frame_test)
y_actual = h2o.H2OFrame(data[['Flux']])

h2o_mean_squared_error(y_actual, y_pred)

In [ ]:
y_pred

In [ ]:
y_actual

In [ ]:
# Convert H2O frames to NumPy arrays
y_pred_np = h2o.as_list(y_pred)
y_actual_np = h2o.as_list(y_actual)
plt.scatter(y_actual_np, y_pred_np, alpha=0.5)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
# Extract data
data = data.dropna()
x = data['Time']
y = y_pred_np
#x_err = [np.abs(data['T_-ve']), data['T_+ve']]
#y_err = [data['Fluxpos'],data['Fluxneg']]

# Create a scatter plot with error bars
#plt.errorbar(x, y, xerr=x_err, yerr=y_err, fmt='o', markersize=5, color='red', ecolor='green', capsize=3)
plt.scatter(x,y,color='red',marker='.',s=10)
# Set x-axis and y-axis to logarithmic scale
plt.xscale('log')
plt.yscale('log')

# Label axes and add a title
plt.xlabel('Time')
plt.ylabel('Flux')
plt.title('Flux vs Time with Error Bars (log-log scale)')

# Show the plot
plt.show()
